In [2]:
import pandas as pd
import numpy as np
import tweepy 
import json

In [ ]:
class PdxCrimeListener(tweepy.streaming.StreamListener):

    def __init__(self):
        self.count = 0
        print("Collecting")

    def on_data(self, data):
        try:
            data = json.loads(data)
            tweet_id = data['id']
            entity = data['user']['name']
            date = pd.to_datetime(data['created_at']).tz_convert('America/')
            year, month, day, hour, minute = date.year,date.month,date.day,date.hour,date.minute
            text = data['text']
            inctime = text[-13:-8]
            cats = text.split('at')[0].strip().split(' - ')
            category = cats[0]
            subcat = 'N/A'
            catstat = 'N/A'
            if len(cats) == 3:
                subcat = cats[1]
                catstat = cats[2]
            elif len(cats) == 2:
                if cats[1] == 'COLD' or cats[1] == 'PRIORITY':
                    catstat = cats[1]
                else:
                    subcat = cats[1]
            lat = data['geo']['coordinates'][0]
            lng = data['geo']['coordinates'][1]
            location = None
            while location == None:
                location = geolocator.reverse('{},{}'.format(lat,lng),timeout=None)
                for i in location.address.split(','):
                    item = i.strip().upper()
                    if item in hoods:
                        location = i.strip()
                        break
                    else:
                        location = 'Unknown'
                        
            cur.execute("INSERT INTO twitter_query (tweet_id, entity, date, inctime, text, lat, lng, location, category, subcat, catstat, year, month, day, hour, minute) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                    (tweet_id, entity, str(date), inctime, text, lat, lng, location, category, subcat, catstat, year, month, day, hour, minute))
            conn.commit()
            self.count+=1
            print(data['text'])
            #uncomment to limit collection
            '''
            if self.count<100:
                return True
            else:
                print(self.count)
                print('Finished')
                return False'''
        except BaseException as e:
            print("Error on_data: %s" % str(e))
            return True

    def on_error(self, status_code):
        print (status_code)
        return True # Don't kill the stream

    def on_timeout(self):
        print ('Timeout...')
        return True # Don't kill the stream

if __name__ == '__main__':

    pdx_listener = PdxCrimeListener()
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

    auth.set_access_token(access_key, access_secret)
    sapi = tweepy.streaming.Stream(auth, pdx_listener)  

    # user ids to listen for
    sapi.filter(follow=(['1602852614','1606472113']))